In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk


###############################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Select a CSV file
infil = pick_file("K:\\Wave_data\\", filterlist="CSV,csv");
println("Selected ",infil)

if uppercase(split(infil, ".")[end]) == "CSV"

    # read .csv file to df
    println("Reading data from file to dataframe")
    flush(stdout)
    df = DataFrame(CSV.File(infil,header=0, delim=","));
    df
    
    # name df headers
    rename!(df,[:Probe1,:Probe2, :Probe3, :Probe4, :Probe5]);
    
    # use only 5632 of the 6000 points == 11 x 512 segments
    
    df = df[1:5632,:]
    
    df.Probe1 =df.Probe1 .- mean(df.Probe1)
    df.Probe2 =df.Probe2 .- mean(df.Probe2)
    df.Probe3 =df.Probe3 .- mean(df.Probe3)
    df.Probe4 =df.Probe4 .- mean(df.Probe4)
    
    # convert Epoch seconds to UTC datetime
    
else

    println("Not able to read this file type at present")
    flush(stdout)
    exit()

end

###############################################
################################################
################################################
##           END OF MAIN PROGRAM
################################################
################################################
################################################

# add time stamps mimicing 2.56Hz spacing
insertcols!(df, 1, :Date=> collect(0:5631) / 100)

p1 = plot(df.Date, df.Probe1, lw=:0.75, lc=:red, label="", title="Probe 1")
p2 = plot(df.Date, df.Probe2, lw=:0.75, lc=:orange, label="", title="Probe 2")
p3 = plot(df.Date, df.Probe3, lw=:0.75, lc=:green, label="", title="Probe 3")
p4 = plot(df.Date, df.Probe4, lw=:0.75, lc=:blue, label="", title="Probe 4")
p5 = plot(df.Date, df.Probe5, lw=:0.75, lc=:violet, label="", title="Probe 5")
    
do_plots = plot(p1, p2, p3, p4, p5, layout=(5,1), xlim=(0,2200/100), size=(1800,1000), framestyle = :box,)

savefig("Time_series_plots.png")

display(do_plots)

In [ ]:
function get_window()
################################################
# calculate Hann window
    return [0.5*(1-cos(2*pi*k/N)) for k in (1:N)]
end    # get_window()


function do_fft(heave, N)
################################################
# calculate the Fourier coefficients vide (5.6.2)
    return([sum([heave[k]*exp(2*pi*-1im*k*l/N) for k in (1:N)]) for l in (1:N)])
end    # do_fft()


function calc_psd(Hl, N)
################################################
# The power spectral density is obtained from the Fourier coefficients    
    PSD = zeros(trunc(Int,N/2))

    for l = 1:trunc(Int,N/2)   
        if (l==1) || (l==trunc(Int,N/2)-1)
            PSD[l] = abs(Hl[l])^2
        else
            PSD[l] = abs(Hl[l])^2+abs(Hl[N-l-1])^2
        end
    end

    # Smooth coefficients vide (5.6.6)
    PSD_smooth = PSD
    [PSD_smooth[i] = PSD[i-1]/4 + PSD[i]/2 + PSD[i+1]/4 for i in (2:trunc(Int,N/2)-1)]

    return PSD_smooth    
end    # calc_psd()


function calc_hm0(Sf,freq)
##########################################    
ax1 = (last(freq) - first(freq)) / (length(freq)-1)

# calc spectral moments m0, m1, m2, m3, and m4
s00 = 0; m0 = 0

for ii in 1:128

    s00 += freq[ii]^0 * Sf[ii];

end

m0 = 0.5*ax1*(first(freq)^0*first(Sf) + 2*s00 + last(freq)^0*last(Sf))

return(4 * m0^0.5)

end    # calc_hm0()


function calc_representative_spectra(frequency,Hm0,Tp,gamma)
##########################################    
    """
    function to calculate representative spectrum based on the Jonswap formula in Tucker and Pitt p.339 (10.3-9a)

    inputs:
        frequency - array of spectral frequencies
        Hm0 - floating point value
        Tp - floating point value
        gamma - floating point value - Peak ehhancement factor (Enter 1 for PM, or 3.3 for Jonswap)

        Typical calls:
        Spectra_PM = calc_representative_spectra(f2, Hm0, Tp, 1.0)
        Spectra_JONSWAP = calc_representative_spectra(f2, Hm0, Tp, 3.3)

    returns:
        Sf - array of representative spectra        
    """

    alpha = 1    # initial Philips constant (will decrease for each iteration required)
    g = 9.81
    fp = 1/Tp    # peak frequency

    hm0 = 99.    # set this to large value (so it will change on first iteration)

    Sf = [];

    while((Hm0 - hm0) <= 0.0005)

        Sf = vcat([alpha*g^2 * (2*pi)^-4 * ff^-5 * exp(-1.25 * (ff/fp)^-4) * gamma^exp(-(ff-fp)^2/(2*0.07^2 * fp^2)) for ff in frequency[findall(<=(fp), frequency)]],
                [alpha*g^2 * (2*pi)^-4 * ff^-5 * exp(-1.25 * (ff/fp)^-4) * gamma^exp(-(ff-fp)^2/(2*0.09^2 * fp^2)) for ff in frequency[findall(>(fp), frequency)]]);
        Sf[1] = 0;

###################################################################################################################################            
###  See discussion at https://stackoverflow.com/questions/44915116/how-to-decide-between-scipy-integrate-simps-or-numpy-trapz  ###
###################################################################################################################################

        ##        hm0 = 4*(np.trapz(Sf, frequency))^0.5    # calculate new Hm0 based on Sf values
        hm0 = calc_hm0(Sf,frequency);   # see https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.simps.html        
        alpha *= 0.95;    # reduce alpha by 5% so iterations approach a limit of 0.0005

    end

    return(Sf)    # calc_representative_spectra()
        
    end    # calc_representative_spectra()


function calc_tp5(f2,Sf)
##########################################
# Calculate Tp5 via Read method
    Sf_max = maximum(Sf)

    numerator = 0; denominator = 0

    Sf_sum = cumsum(Sf.*Sf_max).^5

    for i in 1:length(f2)
        w = Sf[i] / Sf_max
        numerator +=  f2[i] * w^5
        denominator += w^5
    end

    Fp5 = numerator / denominator
    
    return(1/Fp5)    # calc_tp5()

    end    # calc_tp5()


function calculate_frequency_domain_parameters(f2, spectra)
##########################################
# Calculate frequency-domain parameters    
# Calls: calc_tp5()
    ax1 = (last(f2) - first(f2)) / (length(f2)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s00 = 0; s01 = 0; s02 = 0; s03 = 0; s04 = 0;
    m0 = 0; m1 = 0; m2 = 0; m3 = 0; m4 = 0

    for ii in 1:128

        s00 += f2[ii]^0 * spectra[ii]
        s01 += f2[ii]^1 * spectra[ii]
        s02 += f2[ii]^2 * spectra[ii]
        s03 += f2[ii]^3 * spectra[ii]
        s04 += f2[ii]^4 * spectra[ii]

    end

    m0 = 0.5*ax1*(first(f2)^0*first(spectra) + 2*s00 + last(f2)^0*last(spectra))
    m1 = 0.5*ax1*(first(f2)^1*first(spectra) + 2*s01 + last(f2)^1*last(spectra))
    m2 = 0.5*ax1*(first(f2)^2*first(spectra) + 2*s02 + last(f2)^2*last(spectra))
    m3 = 0.5*ax1*(first(f2)^3*first(spectra) + 2*s03 + last(f2)^3*last(spectra))
    m4 = 0.5*ax1*(first(f2)^4*first(spectra) + 2*s04 + last(f2)^4*last(spectra))

    ##println("m0 = ",m0," m1 = ",m1, " m2 = ",m2, " m3 = ",m2, " m4 = ",m4)

    # calc wave parameters Hm0, Hrms, T01, T02, Tc
    Hm0 = 4*sqrt(m0)     # Tucker & Pitt p.32 (2.2-6b)
    Hrms = sqrt(8*m0)    # Goda 2nd. Edition p.262 (9.15)
    T01 = m0/m1          # Tucker & Pitt p.41 Table 2.2 
    T02 = sqrt(m0/m2)    # Tucker & Pitt p.40 (2.3-2)
    Tc = sqrt(m2/m4)     # Tucker & Pitt p.41 Table 2.2 - also see Notes

    # identify spectral peak and frequency as peak
    Fp = f2[argmax(spectra)]
    Tp = 1/Fp
    Tp5 = calc_tp5(f2, spectra)

    # calculate spectral width vide Tucker and Pitt p.85 (5.2-8)
    # Note: for JONSWAP, v = 0.39; for PM, v = 0.425
    v = (m0*m2 / m1^2 - 1)^0.5

    # calculate Skewness vide Tucker and Pitt p.109 (5.5-17)
    Skewness = (m0^2 * m3/m1^3 - 3*v^2 - 1) / v^3;
    
    return(Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness)
    
    @printf("%s; Hm0 = %5.2fm; Hrms = %5.2fm; T01 = %5.2fs; T02 = %5.2fs; Tc = %5.2fs; Tp = %5.2fs; Tp5 = %5.2fs; Skewness = %5.4f",
        Dates.format(start_date, "yyyy-mm-dd HH:MM"),Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness)
    
    end    # calculate_frequency_domain_parameters()


function plot_spectra(data, probe)
################################################
# function to plot spectra calculated by Datawell and from WSEs
    println("Preparing to plot spectra")

    # DWR4 parameters
    data *= 100
    N = 512
    Sample_frequency = 2.56
    dt = 1/Sample_frequency

    # calc window
    w = DSP.Windows.hanning(N)
    w_norm = sqrt(Sample_frequency*sum(w.^2))

    start = 0
    segments = []

    # Do spectral analysis for 11 individual segments of 512 water surface elevations
    for segment in (1:11)
        finish = start+N

        # calculate the Fourier coefficients vide (5.6.2)
        Hl = do_fft(data[start+1:finish].*w./w_norm, N)

        # calculate power spectral density
        Pden = calc_psd(Hl, N);
        segments = [segments;Pden]

        # set start for next segment
        start = start + 256
    end 

    # convert vector to matrix of 17 individual 256-value spectra
    combined_segments = reshape(segments,256,11);
    Pden = mean.(eachrow(combined_segments))

    freqs = [0:1:N/2-1;]*Sample_frequency/N

    ps_w = DSP.Periodograms.welch_pgram(data, 512, 256; onesided=true, nfft=512, fs=Sample_frequency, window=hanning);
    f2 = freq(ps_w);
    Pden2 = power(ps_w);

    freqs = [1:1:N/2;]*Sample_frequency/N

    title_string = probe

    # Calculate the frequency bins for Datawell's datawell_spectra
    datawell_freqs = []

    for i in 0:1:45
        push!(datawell_freqs,0.025 + i*0.005)
    end

    for i in 46:1:79
       push!(datawell_freqs,-0.20 + i*0.01)
    end

    for i in 80:1:99
        push!(datawell_freqs,-0.98 + i*0.02)
    end

    Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness = calculate_frequency_domain_parameters(f2, Pden2)

    # Calculate representative spectra for P-M and JONSWAP
    Spectra_PM = calc_representative_spectra(f2, Hm0, Tp, 1.0);
    Spectra_JONSWAP = calc_representative_spectra(f2, Hm0, Tp, 3.3);
    max_JONSWAP = maximum(Spectra_JONSWAP) * 1.05

    # Get the maximum value on Y-axis
    ##max_y = maximum([maximum(Pden),maximum(Pden2),datawell_max_y[1],maximum(Spectra_JONSWAP)]) * 1.05

    # Plot the representative spectra
    p_spectra = plot(f2[5:end],Spectra_JONSWAP[5:end], lw=2, c=:lightblue, label="JONSWAP spectrum (" * L"\gamma" * " = 3.3)", background_color_legend=:transparent)
    p_spectra = plot!(f2[5:end],Spectra_PM[5:end], lw=2, c=:lightgreen, label="Pierson-Moskowitz spectrum (" * L"\gamma" * " = 1.0)")
                                                
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
    # Add frequency-domain parameters to plot
    x_lim = xlims(p_spectra)[1]; y_lim = ylims(p_spectra)[2]
    p_spectra = annotate!(0.6, max_JONSWAP*0.55, text("Hm0 = " * string(round(Hm0, digits=2)) * "m", 10, :left))
    p_spectra = annotate!(0.6, max_JONSWAP*0.50, text("Hrms = " * string(round(Hrms, digits=2)) * "m", 10, :left))
    p_spectra = annotate!(0.6, max_JONSWAP*0.45, text("T01 = " * string(round(T01, digits=2)) * "s", 10, :left)) 
    p_spectra = annotate!(0.6, max_JONSWAP*0.40, text("T02 = " * string(round(T02, digits=2)) * "s", 10, :left)) 
    p_spectra = annotate!(0.6, max_JONSWAP*0.35, text("Tp = " * string(round(Tp, digits=2)) * "s", 10, :left)) 
    p_spectra = annotate!(0.6, max_JONSWAP*0.30, text("Tp5 = " * string(round(Tp5, digits=2)) * "s", 10, :left)) 
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
    # plot calculated spectra
    p_spectra = plot!(freqs[5:end], Pden[5:end], c="blue", lw=3, label="Calc. spectra\n", 
        fillrange = 0, fillalpha = 0.075, fillcolor = :blue)

    # plot Welch's spectra
    p_spectra = plot!(f2[5:end], Pden2[5:end], c="green", lw=3, label="Welch's spectra\n", 
        fillrange = 0, fillalpha = 0.075, fillcolor = :green)

    # plot Datawell spectra
    #p_spectra = plot!(datawell_freqs,datawell_spectra, c="red", lw=3, label="Datawell spectra",
    #    fillrange = 0, fillalpha = 0.075, fillcolor = :red)

    p_spectra = vline!([1/Tp5; 1/Tp5], lw=1, ls =:dot, c=:red, label="Tp5")

    spec = Plots.plot(p_spectra, layout = (1, 1), size = (1400, 800),
            xlim=(0,0.8), yticks=0:0.05:max_JONSWAP, ytickfontsize=8, ylabel="Spectral Density (m"*L"^2"*"/Hz.)",
            framestyle = :box, fg_legend=:transparent,
            title=title_string, titlefontsize=12,
            leftmargin=7*Plots.mm, rightmargin=5*Plots.mm, topmargin=0*Plots.mm, bottommargin=5*Plots.mm,
            grid=true, gridlinewidth=0.5, gridalpha=1, foreground_color_grid="lightgrey")            

end    # plot_spectra()
                                              
s1 = plot_spectra(df.Probe1, "Probe 1")
s2 = plot_spectra(df.Probe2, "Probe 2")
s3 = plot_spectra(df.Probe3, "Probe 3")
s4 = plot_spectra(df.Probe4, "Probe 4")
s5 = plot_spectra(df.Probe5, "Probe 5")
                                                
spectral_plots = plot(s1, s2, s3, layout=(3,1), size=(1000,1200))
                                                        
savefig(".\\Spectral_plots.png")
                                                                                                       
display(spectral_plots)

In [ ]:
function get_window()
################################################
# calculate Hann window
    return [0.5*(1-cos(2*pi*k/N)) for k in (1:N)]
end    # get_window()


function do_fft(heave, N)
################################################
# calculate the Fourier coefficients vide (5.6.2)
    return([sum([heave[k]*exp(2*pi*-1im*k*l/N) for k in (1:N)]) for l in (1:N)])
end    # do_fft()


function calc_psd(Hl, N)
################################################
# The power spectral density is obtained from the Fourier coefficients    
    PSD = zeros(trunc(Int,N/2))

    for l = 1:trunc(Int,N/2)   
        if (l==1) || (l==trunc(Int,N/2)-1)
            PSD[l] = abs(Hl[l])^2
        else
            PSD[l] = abs(Hl[l])^2+abs(Hl[N-l-1])^2
        end
    end

    # Smooth coefficients vide (5.6.6)
    PSD_smooth = PSD
    [PSD_smooth[i] = PSD[i-1]/4 + PSD[i]/2 + PSD[i+1]/4 for i in (2:trunc(Int,N/2)-1)]

    return PSD_smooth    

end    # calc_psd()


function calc_hm0(Sf,freq)
##########################################    
    ax1 = (last(freq) - first(freq)) / (length(freq)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s00 = 0; m0 = 0

    for ii in 1:128

        s00 += freq[ii]^0 * Sf[ii];

    end

    m0 = 0.5*ax1*(first(freq)^0*first(Sf) + 2*s00 + last(freq)^0*last(Sf))

    return(4 * m0^0.5)

    end    # calc_hm0()


function calc_tp5(f2,Sf)
##########################################
# Calculate Tp5 via Read method
    Sf_max = maximum(Sf)

    numerator = 0; denominator = 0

    Sf_sum = cumsum(Sf.*Sf_max).^5

    for i in 1:length(f2)
        w = Sf[i] / Sf_max
        numerator +=  f2[i] * w^5
        denominator += w^5
    end

    Fp5 = numerator / denominator
    
    return(1/Fp5)    # calc_tp5()

    end    # calc_tp5()


function calculate_frequency_domain_parameters(f2, spectra)
##########################################
# Calculate frequency-domain parameters    
# Calls: calc_tp5()
    ax1 = (last(f2) - first(f2)) / (length(f2)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s00 = 0; s01 = 0; s02 = 0; s03 = 0; s04 = 0;
    m0 = 0; m1 = 0; m2 = 0; m3 = 0; m4 = 0

    for ii in 1:128

        s00 += f2[ii]^0 * spectra[ii]
        s01 += f2[ii]^1 * spectra[ii]
        s02 += f2[ii]^2 * spectra[ii]
        s03 += f2[ii]^3 * spectra[ii]
        s04 += f2[ii]^4 * spectra[ii]

    end

    m0 = 0.5*ax1*(first(f2)^0*first(spectra) + 2*s00 + last(f2)^0*last(spectra))
    m1 = 0.5*ax1*(first(f2)^1*first(spectra) + 2*s01 + last(f2)^1*last(spectra))
    m2 = 0.5*ax1*(first(f2)^2*first(spectra) + 2*s02 + last(f2)^2*last(spectra))
    m3 = 0.5*ax1*(first(f2)^3*first(spectra) + 2*s03 + last(f2)^3*last(spectra))
    m4 = 0.5*ax1*(first(f2)^4*first(spectra) + 2*s04 + last(f2)^4*last(spectra))

    ##println("m0 = ",m0," m1 = ",m1, " m2 = ",m2, " m3 = ",m2, " m4 = ",m4)

    # calc wave parameters Hm0, Hrms, T01, T02, Tc
    Hm0 = 4*sqrt(m0)     # Tucker & Pitt p.32 (2.2-6b)
    Hrms = sqrt(8*m0)    # Goda 2nd. Edition p.262 (9.15)
    T01 = m0/m1          # Tucker & Pitt p.41 Table 2.2 
    T02 = sqrt(m0/m2)    # Tucker & Pitt p.40 (2.3-2)
    Tc = sqrt(m2/m4)     # Tucker & Pitt p.41 Table 2.2 - also see Notes

    # identify spectral peak and frequency as peak
    Fp = f2[argmax(spectra)]
    Tp = 1/Fp
    Tp5 = calc_tp5(f2, spectra)

    # calculate spectral width vide Tucker and Pitt p.85 (5.2-8)
    # Note: for JONSWAP, v = 0.39; for PM, v = 0.425
    v = (m0*m2 / m1^2 - 1)^0.5

    # calculate Skewness vide Tucker and Pitt p.109 (5.5-17)
    Skewness = (m0^2 * m3/m1^3 - 3*v^2 - 1) / v^3;
    
    return(Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness)
    
#    @printf("%s; Hm0 = %5.2fm; Hrms = %5.2fm; T01 = %5.2fs; T02 = %5.2fs; Tc = %5.2fs; Tp = %5.2fs; Tp5 = %5.2fs; Skewness = %5.4f",
#        Dates.format(start_date, "yyyy-mm-dd HH:MM"),Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness)
    
    end    # calculate_frequency_domain_parameters()


function calc_representative_spectra(frequency,Hm0,Tp,gamma)
##########################################    
    """
    function to calculate representative spectrum based on the Jonswap formula in Tucker and Pitt p.339 (10.3-9a)

    inputs:
        frequency - array of spectral frequencies
        Hₘ₀ - floating point value
        Tₚ - floating point value
        gamma - floating point value - Peak ehhancement factor (Enter 1 for PM, or 3.3 for Jonswap)

        Typical calls:
        Spectra_PM = calc_representative_spectra(frequency, Hₘ₀, Tₚ, 1.0)
        Spectra_JONSWAP = calc_representative_spectra(frequency, Hₘ₀, Tₚ, 3.3)

    returns:
        Sf - array of representative spectra        
    """

    alpha = 1    # initial Philips constant (will decrease for each iteration required)
    g = 9.81
    fp = 1/Tp    # peak frequency

    hm0 = 99.    # set this to large value (so it will change on first iteration)

    Sf = [];

    while((Hm0 - hm0) <= 0.0005)
        
        Sf = vcat([alpha*g^2 * (2*pi)^-4 * ff^-5 * exp(-1.25 * (ff/fp)^-4) * gamma^exp(-(ff-fp)^2/(2*0.07^2 * fp^2)) for ff in frequency[findall(<=(fp), frequency)]],
                [alpha*g^2 * (2*pi)^-4 * ff^-5 * exp(-1.25 * (ff/fp)^-4) * gamma^exp(-(ff-fp)^2/(2*0.09^2 * fp^2)) for ff in frequency[findall(>(fp), frequency)]]);
        Sf[1] = 0;

##        Hm0 = 4*(np.trapz(Sf, frequency))^0.5    # calculate new Hₘ₀ based on Sf values
        hm0 = calc_hm0(Sf,frequency);   # see https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.simps.html        
        alpha *= 0.95;    # reduce alpha by 5% so iterations approach a limit of 0.0005

    end

    return(Sf)    # calc_representative_spectra()
        
end    # calc_representative_spectra()


function calc_fft(data)
##################################
    
    # Segment length
    N = 512
    Sample_frequency = 100.0
    dt = 1/Sample_frequency

    # calc window
    w = DSP.Windows.hanning(N)
    w_norm = sqrt(Sample_frequency*sum(w.^2))

    start = 0
    segments = []

    # Do spectral analysis for 11 individual segments of 512 water surface elevations
    for segment in (1:11)
        finish = start+N

        # calculate the Fourier coefficients vide (5.6.2)
        Hl = do_fft(data[start+1:finish].*w./w_norm, N)

        # calculate power spectral density
        Pden = calc_psd(Hl, N);
        segments = [segments;Pden]

        # set start for next segment
        start = start + 256
    end 

    # convert vector to matrix of 17 individual 256-value spectra
    combined_segments = reshape(segments,256,11);
    Pden = mean.(eachrow(combined_segments))

    freqs = [0:1:N/2-1;]*Sample_frequency/N
    
    return(freqs, Pden)
    
end    # calc_fft()
                    

function do_spectral_plot(spectra, df, i, probes)
######################################################

    probe = eval(Meta.parse("df."*probes[i]))
    probe_name = string.(propertynames(df)[2:end])[i]
    
    welch_spectra = DSP.Periodograms.welch_pgram(probe, 512, 256; onesided=true, nfft=512, fs=100, window=hanning)
    freqs = freq(welch_spectra)
    global powers = power(welch_spectra)

    Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness = calculate_frequency_domain_parameters(freqs,powers)
    
    Spectra_PM = calc_representative_spectra(freqs, Hm0, Tp, 1.0)
    Spectra_JONSWAP = calc_representative_spectra(freqs, Hm0, Tp, 3.3)
    
    max_power = max(maximum(Spectra_JONSWAP),maximum(powers))*1.05
    
    p1 = plot(freqs,powers, title=probe_names[i], lw=:2, lc=:green, fillrange = 0, fillalpha = 0.075, fillcolor = :green, 
        label="Welch's method\n", xlim=(0,20), xticks=0:1:20, ylim=(0,max_power))
    
    p_spectra = vline!([1/Tp5; 1/Tp5], lw=1, ls =:dot, c=:red, label="Tp5")

    # Add frequency-domain parameters to plot
    x_lim = xlims(p1)[1]; y_lim = ylims(p1)[2]
    p1 = annotate!(12, max_power*0.50, text("Hm0 = " * string(round(Hm0, digits=3)) * "m", 8, :left))
    p1 = annotate!(12, max_power*0.45, text("Hrms = " * string(round(Hrms, digits=3)) * "m", 8, :left))
    p1 = annotate!(12, max_power*0.40, text("T01 = " * string(round(T01, digits=3)) * "s", 8, :left)) 
    p1 = annotate!(12, max_power*0.35, text("T02 = " * string(round(T02, digits=3)) * "s", 8, :left)) 
    p1 = annotate!(12, max_power*0.30, text("Tp = " * string(round(Tp, digits=3)) * "s", 8, :left))
    p1 = annotate!(12, max_power*0.25, text("Tp5 = " * string(round(Tp5, digits=3)) * "s", 8, :left))
            
    p1 = plot!(freqs,Spectra_PM, lw=3, c=:lightgreen, label="Pierson-Moskowitz spectrum (" * L"\gamma" * " = 1.0)\n")
    p1 = plot!(freqs,Spectra_JONSWAP, lw=3, c=:lightblue, label="JONSWAP spectrum (" * L"\gamma" * " = 3.3)\n", background_color_legend=:transparent)
    p1 = plot!(calc_fft(probe), c="blue", lw=3, label="Calc. spectra", fillrange = 0, fillalpha = 0.075, fillcolor = :blue)
    
    push!(spectra, p1)

end    # do_spectral_plot()
                        
                        
spectra = []
probe_names = string.(propertynames(df)[2:end])

for i in 1:length(probe_names)

    do_spectral_plot(spectra, df, i, probe_names)

end

col_no = Int(round((length(spectra) / 2) + 0.5))

plot_all = plot(spectra..., size=(1800, 400*col_no), layout=(col_no,2),
    fg_legend=:transparent, bg_legend=:transparent,
    leftmargin=7.5*Plots.mm, rightmargin=0*Plots.mm, bottommargin=0*Plots.mm, topmargin=0*Plots.mm, framestyle = :box)